In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [3]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer, StandardScaler, MinMaxScaler
from sklearn.preprocessing import RobustScaler, MaxAbsScaler

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier

imputers = [('imputer', Imputer())]
scalers =  [('std_scaler', StandardScaler()), ('minmax_scaler', MinMaxScaler()),
            ('maxabs_scaler', MaxAbsScaler()), ('robust_scaler', RobustScaler())];
pcas = [('pca_4comps', PCA(n_components = 4))]
estimators = [('booster', GradientBoostingClassifier())]
random_state = np.random.RandomState(0)
clf = Pipeline(imputers + scalers + pcas + estimators)

clf.fit(X, Y)

Pipeline(steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('std_scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('minmax_scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('maxabs_scaler', MaxAbsScaler(copy=True)), ('robust_scaler', ...=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False))])

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model" # "https://sklearn2sql.herokuapp.com/"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [6]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:120000])

WITH "ADS_imp_1_OUT" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_0" IS NULL) THEN 5.84333333333 ELSE "ADS"."Feature_0" END AS imputer_output_2, CASE WHEN ("ADS"."Feature_1" IS NULL) THEN 3.054 ELSE "ADS"."Feature_1" END AS imputer_output_3, CASE WHEN ("ADS"."Feature_2" IS NULL) THEN 3.75866666667 ELSE "ADS"."Feature_2" END AS imputer_output_4, CASE WHEN ("ADS"."Feature_3" IS NULL) THEN 1.19866666667 ELSE "ADS"."Feature_3" END AS imputer_output_5 
FROM "INPUT_DATA" AS "ADS"), 
"ADS_std_2_OUT" AS 
(SELECT "ADS_imp_1_OUT"."KEY" AS "KEY", (CAST("ADS_imp_1_OUT".imputer_output_2 AS FLOAT) - 5.84333333333) / 0.825301291785 AS std_scaler_output_2, (CAST("ADS_imp_1_OUT".imputer_output_3 AS FLOAT) - 3.054) / 0.432146580071 AS std_scaler_output_3, (CAST("ADS_imp_1_OUT".imputer_output_4 AS FLOAT) - 3.75866666667) / 1.75852918341 AS std_scaler_output_4, (CAST("ADS_imp_1_OUT".imputer_output_5 AS FLOAT) - 1.19866666667) / 0.760612618588 AS std_scaler_output_5 
FROM "ADS_imp_1_OUT"), 


## Execute the SQL Code

In [7]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [8]:
sql_output = pd.read_sql(lSQL , conn);
sql_output = sql_output.sort_values(by='KEY').reset_index(drop=True)

In [9]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,-5.847731,-2.386357,5.636589,0.000010,0.000328,0.999662,-11.484658,-8.023285,-0.000338,2
74,74,-5.949027,4.889878,-4.534741,0.000020,0.999900,0.000081,-10.839005,-0.000100,-9.424720,1
9,9,7.360130,-0.304061,-3.975415,0.999519,0.000469,0.000012,-0.000481,-7.664672,-11.336026,0
88,88,-5.896455,3.123427,-3.313537,0.000121,0.998281,0.001599,-9.021603,-0.001721,-6.438685,1
25,25,7.360130,-0.864936,-3.975415,0.999720,0.000268,0.000012,-0.000280,-8.225346,-11.335825,0
5,5,7.300883,-1.060926,-4.427003,0.999758,0.000234,0.000008,-0.000242,-8.362051,-11.728128,0
48,48,7.300883,-0.833054,-4.669319,0.999700,0.000293,0.000006,-0.000300,-8.134236,-11.970501,0
117,117,-5.883144,-1.628549,4.584161,0.000028,0.002000,0.997972,-10.469335,-6.214740,-0.002030,2
83,83,-5.864954,2.933755,-1.269846,0.000149,0.985132,0.014720,-8.813688,-0.014980,-4.218580,1
105,105,-5.873120,-3.348146,3.728675,0.000068,0.000844,0.999089,-9.602706,-7.077733,-0.000912,2


In [10]:
sql_output.Decision.value_counts()

2    50
1    50
0    50
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.000010,0.000328,0.999662,-11.484658,-8.023285,-0.000338,2
74,74,NaN,NaN,NaN,0.000020,0.999900,0.000081,-10.839005,-0.000100,-9.424720,1
9,9,NaN,NaN,NaN,0.999519,0.000469,0.000012,-0.000481,-7.664672,-11.336026,0
88,88,NaN,NaN,NaN,0.000121,0.998281,0.001599,-9.021603,-0.001721,-6.438685,1
25,25,NaN,NaN,NaN,0.999720,0.000268,0.000012,-0.000280,-8.225346,-11.335825,0
5,5,NaN,NaN,NaN,0.999758,0.000234,0.000008,-0.000242,-8.362051,-11.728128,0
48,48,NaN,NaN,NaN,0.999700,0.000293,0.000006,-0.000300,-8.134236,-11.970501,0
117,117,NaN,NaN,NaN,0.000028,0.002000,0.997972,-10.469335,-6.214740,-0.002030,2
83,83,NaN,NaN,NaN,0.000149,0.985132,0.014720,-8.813688,-0.014980,-4.218580,1
105,105,NaN,NaN,NaN,0.000068,0.000844,0.999089,-9.602706,-7.077733,-0.000912,2


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12, random_state=1960)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
114,114,NaN,NaN,NaN,0.000010,0.000328,0.999662,-11.484658,-8.023285,-0.000338,...,-5.847731,-2.386357,5.636589,0.000010,0.000328,0.999662,-11.484658,-8.023285,-0.000338,2
74,74,NaN,NaN,NaN,0.000020,0.999900,0.000081,-10.839005,-0.000100,-9.424720,...,-5.949027,4.889878,-4.534741,0.000020,0.999900,0.000081,-10.839005,-0.000100,-9.424720,1
9,9,NaN,NaN,NaN,0.999519,0.000469,0.000012,-0.000481,-7.664672,-11.336026,...,7.360130,-0.304061,-3.975415,0.999519,0.000469,0.000012,-0.000481,-7.664672,-11.336026,0
88,88,NaN,NaN,NaN,0.000121,0.998281,0.001599,-9.021603,-0.001721,-6.438685,...,-5.896455,3.123427,-3.313537,0.000121,0.998281,0.001599,-9.021603,-0.001721,-6.438685,1
25,25,NaN,NaN,NaN,0.999720,0.000268,0.000012,-0.000280,-8.225346,-11.335825,...,7.360130,-0.864936,-3.975415,0.999720,0.000268,0.000012,-0.000280,-8.225346,-11.335825,0
5,5,NaN,NaN,NaN,0.999758,0.000234,0.000008,-0.000242,-8.362051,-11.728128,...,7.300883,-1.060926,-4.427003,0.999758,0.000234,0.000008,-0.000242,-8.362051,-11.728128,0
48,48,NaN,NaN,NaN,0.999700,0.000293,0.000006,-0.000300,-8.134236,-11.970501,...,7.300883,-0.833054,-4.669319,0.999700,0.000293,0.000006,-0.000300,-8.134236,-11.970501,0
117,117,NaN,NaN,NaN,0.000028,0.002000,0.997972,-10.469335,-6.214740,-0.002030,...,-5.883144,-1.628549,4.584161,0.000028,0.002000,0.997972,-10.469335,-6.214740,-0.002030,2
83,83,NaN,NaN,NaN,0.000149,0.985132,0.014720,-8.813688,-0.014980,-4.218580,...,-5.864954,2.933755,-1.269846,0.000149,0.985132,0.014720,-8.813688,-0.014980,-4.218580,1
105,105,NaN,NaN,NaN,0.000068,0.000844,0.999089,-9.602706,-7.077733,-0.000912,...,-5.873120,-3.348146,3.728675,0.000068,0.000844,0.999089,-9.602706,-7.077733,-0.000912,2


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_0_sql,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql
